In [1]:
###
###  First, check your TF version and the hardware enviorment
###
import tensorflow as tf
from tensorflow.python.client import device_lib

print ("Using Tensorflow: ", tf.__version__)
print ( device_lib.list_local_devices() )


/home/lincy/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


('Using Tensorflow: ', '1.6.0')
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16251905635672263848
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 108331008
locality {
  bus_id: 1
}
incarnation: 18089538405850170452
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:03:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 241565696
locality {
  bus_id: 2
}
incarnation: 8270135198320549881
physical_device_desc: "device: 1, name: GeForce GTX 1060 6GB, pci bus id: 0000:84:00.0, compute capability: 6.1"
]


In [2]:
### First, check GPU status if you have a GPU 
!nvidia-smi

Fri Jun  1 00:41:12 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:03:00.0 Off |                  N/A |
| 28%   28C    P2    28W / 120W |   5854MiB /  6078MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 106...  Off  | 00000000:84:00.0 Off |                  N/A |
| 28%   27C    P2    27W / 120W |   5854MiB /  6078MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
###
### Variables, Constant, Placeholders
###
import tensorflow as tf
tf.reset_default_graph()

## create a variable of rank-3 zero tensor  
# GraphKeys can be such as:
#   GLOBAL_VARIABLES
#   LOCAL_VARIABLES
#   TRAINABLE_VARIABLES, or control by "trainable=False"
#
var = tf.get_variable("var", shape=[2, 3], dtype=tf.float32, 
         initializer=tf.zeros_initializer, collections=[tf.GraphKeys.LOCAL_VARIABLES])

with tf.variable_scope("ABC"):
    a = tf.add(3, 5)
    a = tf.cast(a, tf.float32) 

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print (sess.run(a) )
print (sess.run( tf.sin(a) ))

sess.run(tf.local_variables_initializer())
var = (var + 1.0 )*5
print (sess.run(var))



sess.close()


8.0
0.98935825
[[5. 5. 5.]
 [5. 5. 5.]]


In [4]:
### For advanced feature, we can specify which device to be used. 
### This is only for futher development of distributed training.
###
### We can program TF just like a complex MPI program with C.
###  
import tensorflow as tf
from tensorflow.python.client import device_lib

# Creates a graph.
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2,3],name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3,2],name='b')
    c = tf.matmul(a, b)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print (sess.run(c) )

[[22. 28.]
 [49. 64.]]


In [5]:
###
###  Build a simple a perceptron model -- logistic regression (sigmoid)
###
import tensorflow as tf
tf.reset_default_graph()

IN_DIM = 3
OUT_DIM = 1
x = tf.placeholder(tf.float32, [None, IN_DIM])
y = tf.placeholder(tf.float32, [None, OUT_DIM])

W  = tf.Variable(tf.zeros([IN_DIM, OUT_DIM]), name="W")
b  = tf.Variable(tf.zeros([OUT_DIM]),         name="b")

output  = tf.sigmoid(tf.matmul(x, W) + b)

xdata = [  [1.,2.,3.],
           [1.,2.,3.]   ]  ## input data with batch size = 2
#xdata2 = tf.ones_like(xdata)



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out = sess.run([output], feed_dict={x: xdata}  )   #  2 outputs 
    print (out)

[array([[0.5],
       [0.5]], dtype=float32)]
